In [47]:
pwd

'/home/wsuser/work'

In [48]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.metrics import Accuracy
import numpy as np

In [49]:
xcep = Xception(include_top=False, weights="imagenet", input_shape=[224,224,3])

In [50]:
pwd

'/home/wsuser/work'

In [51]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='uUDHjhEV9PPH4hfR_H8J94m4LXyQTEzsD0EetvLqv9ZW',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'yogaposedetection-donotdelete-pr-mhehiyzaxdodcr'
object_key = 'Dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [52]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [53]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset')

In [54]:
for layer in xcep.layers:
  layer.trainable = False

In [55]:
x = Flatten()(xcep.output)

In [56]:
prediction  = Dense(5, activation='softmax')(x)

In [57]:
model = Model(inputs = xcep.input, outputs=prediction)


In [58]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

In [59]:
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = 'Accuracy')

In [60]:
data = ImageDataGenerator(horizontal_flip=True, shear_range=0.2,zoom_range=0.2,rescale=1./255, validation_split=0.3)

In [61]:
training_set = data.flow_from_directory(r'/home/wsuser/work/Dataset',
                                           target_size=(224,224),
                                           batch_size=16,
                                           subset='training',
                                           class_mode='categorical')

Found 694 images belonging to 5 classes.


In [62]:
testing_set = data.flow_from_directory(r'/home/wsuser/work/Dataset',
                                        target_size=(224,224),
                                           batch_size=16,
                                           subset='validation',
                                           class_mode='categorical')

Found 294 images belonging to 5 classes.


In [63]:
len(training_set), len(testing_set)

(44, 19)

In [64]:
model.fit(training_set,
          validation_data = testing_set,
          epochs =15,
          steps_per_epoch=len(training_set)//16,
          validation_steps = len(testing_set)//16)


Epoch 1/15
2/2 [==============================] - 16s 9s/step - loss: 8.1191 - Accuracy: 0.2812 - val_loss: 9.2052 - val_Accuracy: 0.1250
Epoch 2/15
2/2 [==============================] - 11s 7s/step - loss: 7.9404 - Accuracy: 0.2812 - val_loss: 1.5266 - val_Accuracy: 0.5625
Epoch 3/15
2/2 [==============================] - 12s 8s/step - loss: 3.0674 - Accuracy: 0.4375 - val_loss: 5.6036 - val_Accuracy: 0.3750
Epoch 4/15
2/2 [==============================] - 11s 8s/step - loss: 2.0631 - Accuracy: 0.6562 - val_loss: 1.5448 - val_Accuracy: 0.8125
Epoch 5/15
2/2 [==============================] - 8s 7s/step - loss: 2.6351 - Accuracy: 0.6818 - val_loss: 4.5931 - val_Accuracy: 0.4375
Epoch 6/15
2/2 [==============================] - 12s 8s/step - loss: 2.5828 - Accuracy: 0.7188 - val_loss: 1.5984 - val_Accuracy: 0.8125
Epoch 7/15
2/2 [==============================] - 9s 6s/step - loss: 3.3535 - Accuracy: 0.5938 - val_loss: 2.9529 - val_Accuracy: 0.7500
Epoch 8/15
2/2 [====================

In [66]:
model.save('xcep_yoga.h5')

In [67]:
!tar -zcvf image-classification-model_new.tgz xcep_yoga.h5

xcep_yoga.h5


In [68]:
ls -1

Dataset/
image-classification-model_new.tgz
model2.tar.gb
xcep_yoga.h5


In [69]:
!pip install watson-machine-learning-client --upgrade

In [70]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                   "url":"https://us-south.ml.cloud.ibm.com",
                   "apikey":"6-H_ym4Gk9LdmTZDhAcKn8WQV7Mcp0pIULEXfova-wBV"
                  }
client = APIClient(wml_credentials)

In [71]:
client = APIClient(wml_credentials)

In [72]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item ['entity']["name"] == space_name)['metadata']['id'])

In [73]:
space_uid = guid_from_space_name(client, 'models')
print("Space UID = " + space_uid)

Space UID = 8b12396a-f7b3-426e-b5b8-dcc11d1b2a44


In [74]:
client.set.default_space(space_uid)

'SUCCESS'

In [75]:
client.software_specifications.list(100
                                   )

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [76]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [78]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"CNN-model",
client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


In [108]:
model_id

'7a9d000b-a2ca-4fd8-95ce-7a8eb4dc8d63'

In [110]:
client.repository.download(model_id,'model0.tar.gb')

Successfully saved model content to file: 'model0.tar.gb'


'/home/wsuser/work/model0.tar.gb'